In [157]:
from datetime import date, datetime, time,timedelta
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

# from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import recall_score

In [182]:
df=pd.read_excel(r'C:\Users\User\Documents\Обучающая выборка3.xlsx')

In [183]:
# Фунцкия для кросс-валидации дерева
def dec_tree(Y,depth,crit,X):

    clf = DecisionTreeClassifier(max_depth=depth,
                                 criterion=crit,
                                    class_weight='balanced')
    
    #обучаемся на тест_трейне, для оценки точности методов
    a=cross_val_score(clf, X,Y, cv=5, scoring='recall')

    print([i,crit,depth,a.mean()])

In [184]:
# Граничные состояния сна

cur_day=date.today()- timedelta(days=1)

norm_to_sleep= datetime.combine(cur_day, time(22, 0))

late_to_sleep= datetime.combine(date.today(), time(0, 30))

norm_wakeup= datetime.combine(date.today(), time(5, 0))

late_wakeup= datetime.combine(date.today(), time(8, 0))

In [185]:
# Колонки для выкидывания
drop_col=["Этнос","Национальность",'Туберкулез легких ','ВИЧ/СПИД','Семья','Время засыпания','Время пробуждения']

zcol=['Нет заболеваний','Артериальная гипертензия','ОНМК','Стенокардия, ИБС, инфаркт миокарда',
      'Сердечная недостаточность','Прочие заболевания сердца']

col_z01=['Пол','Образование', 'Профессия', 'Выход на пенсию','Сахарный диабет', 'Гепатит',
'Хроническое заболевание легких','Регулярный прим лекарственных средств', 'Травмы за год', 
'Переломы','Статус Курения', 'Возраст курения', 'Сигарет в день','Частота пасс кур', 
'Алкоголь', 'Возраст алког','засыпание', 'просыпание', 'сон_часы','Сон после обеда']

xcol_z01=['x_'+str(x+1) for x in range(len(col_z01))]

zcol_z01=['z_'+str(x) for x in range(len(zcol))]

# Словарь переименования col_z0_1
xdic_z01=dict(zip(col_z01, xcol_z01))
zdic_z01=dict(zip(zcol, zcol_z01))

In [186]:
# Колонки для замены пустот 0
# empty_col=["Частота пасс кур"]

# Словарь кодировки категорий в числа
dict_replace=[
['Пол',{'М':1,'Ж':0}],
    
['Религия',{'Христианство':1,'Ислам':1,'Атеист / агностик':0,'Нет':0}],  
    
['Профессия',{'ведение домашнего хозяйства':0,
              'низкоквалифицированные работники':1,
              'операторы и монтажники установок и машинного оборудования':2,
              'техники и младшие специалисты':2,
              'ремесленники и представители других отраслей промышленности':2,
              'служащие':2,
              'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры':2,
              'вооруженные силы':2,
              'квалифицированные работники сельского хозяйства и рыболовного':2,
              'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков':2,
              'дипломированные специалисты':3}],
    
['Статус Курения',{'Никогда не курил(а)':0,
                   'Никогда не курил':0,
                   'Бросил(а)':1,
                   'Бросил':1,
                  'Курит':2}],
    
['Частота пасс кур',{'не менее 1 раза в день':0,
                                   '1-2 раза в неделю':1,
                                  '2-3 раза в день':0,
                               '3-6 раз в неделю':0}],

['Алкоголь',{"никогда не употреблял":0,
                "ранее употреблял":1,
                "употребляю в настоящее время":2}],    
    
# 8.	Алкоголь (0 – никогда, 1 – употреблял, 2 – употребляю)
['Образование',{"2 - начальная школа":2,
                "3 - средняя школа / закон.среднее / выше среднего":3,
                "4 - профессиональное училище":4,
                "5 - ВУЗ":5}]
]

In [187]:
def preproc_data(df,dict_replace,norm_to_sleep,late_to_sleep,norm_wakeup,late_wakeup,drop_col):

    # Преобразование времени в датувремя
    df['Время засыпания']=pd.to_datetime(df['Время засыпания'].astype(str))
    df['Время пробуждения']=pd.to_datetime(df['Время пробуждения'].astype(str))

    df.loc[df['Время засыпания']>
                df['Время пробуждения'],
                ['Время засыпания']]=df.loc[df['Время засыпания']>
                                                 df['Время пробуждения'],
                                                 ['Время засыпания']]-pd.DateOffset(days=1)

    # 9.	Засыпание (0 -до 22:30, 2 – после полуночи)
    df['засыпание']=0
    df.loc[(df['Время засыпания']>=norm_to_sleep)&(df['Время засыпания']<=late_to_sleep),'засыпание']=1
    df.loc[(df['Время засыпания']>late_to_sleep),'засыпание']=2

    # 10.	Просыпание ( 0 -до 5 ч, 2 – после 8)
    df['просыпание']=0
    df.loc[(df['Время пробуждения']>norm_wakeup)&(df['Время пробуждения']<=late_wakeup),'просыпание']=1
    df.loc[(df['Время пробуждения']>late_wakeup),'просыпание']=2

    # Часов сна
    df['сон_часы']=(df['Время пробуждения']-df['Время засыпания']).astype('timedelta64[m]')/60

    # Преобразования Семьи в колонки
    df=df.assign(В_БРАКЕ=0,вдовцы=0,не_был=0,Развод=0,Граждан=0) # добавляем доп колонки

    # Записываем 1
    df.loc[df['Семья'].isin(['в браке в настоящее время','раздельное проживание (официально не разведены)']),'В_БРАКЕ']=1
    df.loc[df['Семья']=='вдовец / вдова','вдовцы']=1
    df.loc[df['Семья'].isin(['никогда не был(а) в браке','никогда не был в браке']),'не_был']=1
    df.loc[df['Семья']=='в разводе','Развод']=1
    df.loc[df['Семья']=='гражданский брак / проживание с партнером','Граждан']=1

    # Преобразования категорий в числа из словаря
    for it_cat in dict_replace:
        df[it_cat[0]]=df[it_cat[0]].map(it_cat[1])

    # Выбрасывание лишних колонок
    df=df.drop(drop_col,axis=1)
    
    #Заполнение пустот 0
    df=df.fillna(0)
    
        
    return(df)

In [188]:
df=preproc_data(df,dict_replace,norm_to_sleep,late_to_sleep,norm_wakeup,late_wakeup,drop_col)

In [189]:
# Модель 2
df_z01=df.rename(columns=zdic_z01)

Y_col=['z_1','z_2','z_3','z_4','z_5']

X_col=df_z01.drop(Y_col+['ID'],axis=1).columns

X,Y=df_z01[X_col].fillna(0),df_z01[Y_col].fillna(0)

test_data=preproc_data(pd.read_excel(r'C:\Users\User\Desktop\Python csv\Heart\test.xlsx'),
                       dict_replace,norm_to_sleep,late_to_sleep,norm_wakeup,late_wakeup,
                       drop_col).set_index('ID')

In [190]:
# ['z_1', 'entropy', 3, 0.8204494382022472]
# ['z_2', 'gini', 3, 0.6083333333333333]
# ['z_3', 'gini', 5, 0.7532608695652174]
# ['z_4', 'gini', 4, 0.7389473684210526]
# ['z_5', 'gini', 4, 0.6176470588235294]

In [191]:
# Выбираем параметры для дерева исходя из результатов с кросс-валидацией

for i in Y.columns:

    for it_depth in [3,4,5,6]:

        for it_crit in ['gini', 'entropy']:
      
            dec_tree(Y[i],it_depth,it_crit,X)

['z_1', 'gini', 3, 0.8114606741573034]
['z_1', 'entropy', 3, 0.8204494382022472]
['z_1', 'gini', 4, 0.7645443196004993]
['z_1', 'entropy', 4, 0.7824968789013733]
['z_1', 'gini', 5, 0.7353058676654183]
['z_1', 'entropy', 5, 0.7532084893882647]
['z_1', 'gini', 6, 0.7308364544319599]
['z_1', 'entropy', 6, 0.757752808988764]
['z_2', 'gini', 3, 0.5333333333333333]
['z_2', 'entropy', 3, 0.5333333333333333]
['z_2', 'gini', 4, 0.4138888888888889]
['z_2', 'entropy', 4, 0.43888888888888894]
['z_2', 'gini', 5, 0.39166666666666666]
['z_2', 'entropy', 5, 0.4916666666666667]
['z_2', 'gini', 6, 0.3416666666666667]
['z_2', 'entropy', 6, 0.4166666666666667]
['z_3', 'gini', 3, 0.7528985507246377]
['z_3', 'entropy', 3, 0.7536231884057971]
['z_3', 'gini', 4, 0.7532608695652174]
['z_3', 'entropy', 4, 0.7199275362318841]
['z_3', 'gini', 5, 0.7365942028985508]
['z_3', 'entropy', 5, 0.7199275362318841]
['z_3', 'gini', 6, 0.6420289855072465]
['z_3', 'entropy', 6, 0.5833333333333333]
['z_4', 'gini', 3, 0.623157

In [192]:
# Выбранные параметры
# 'z_1', 'entropy', 3, 0.8204494382022472
# 'z_2', 'gini', 3, 0.6083333333333333
# 'z_3', 'gini', 5, 0.7532608695652174
# 'z_4', 'gini', 4, 0.7389473684210526
# 'z_5', 'gini', 4, 0.6176470588235294
# 'z_2' и 'z_5' низкие для них испольую RandomForest

In [193]:
# z1
col='z_1'

clf = DecisionTreeClassifier(max_depth=3,
                            criterion='entropy',
                            class_weight='balanced',
                                max_features='sqrt')

clf.fit(X,Y[col])

test_data[col]=[x[1]>0.5 for x in clf.predict_proba(test_data[X.columns])]

importance = clf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
    if v>0:
        print('{}: {}'.format(X.columns[i],round(v,4)))

Гепатит: 0.0145
Бронжиальная астма: 0.0198
Регулярный прим лекарственных средств: 0.859
Возраст курения: 0.0255
Сигарет в день: 0.0053
сон_часы: 0.0163
вдовцы: 0.0596


In [194]:
# z2
# Дерево показало низко
col='z_2'

clf=RandomForestClassifier(max_depth=3,n_estimators=10,random_state=32,class_weight = 'balanced') #z4 0.5245

#обучаемся на всем множестве данных
clf.fit(X,Y[col])

test_data[col]=[x[1]>0.5 for x in clf.predict_proba(test_data[X.columns])]

importance = clf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
    if v>0.0001:
        print('{}: {}'.format(X.columns[i],round(v,4)))

Пол: 0.0468
Религия: 0.0087
Образование: 0.0609
Профессия: 0.0049
Вы работаете?: 0.1189
Выход на пенсию: 0.025
Прекращение работы по болезни: 0.0341
Сахарный диабет: 0.0312
Гепатит: 0.0364
Хроническое заболевание легких: 0.015
Регулярный прим лекарственных средств: 0.0892
Переломы: 0.0718
Статус Курения: 0.0399
Возраст курения: 0.0765
Сигарет в день: 0.0217
Частота пасс кур: 0.0374
Алкоголь: 0.0057
Возраст алког: 0.0364
Сон после обеда: 0.0531
Спорт, клубы: 0.0225
Религия, клубы: 0.0036
засыпание: 0.0358
просыпание: 0.0237
сон_часы: 0.0532
В_БРАКЕ: 0.0277
Граждан: 0.02


In [195]:
# z3
col='z_3'

clf = DecisionTreeClassifier(max_depth=3,
                            criterion='gini',
                            class_weight='balanced',
                                max_features='sqrt')

clf.fit(X,Y[col])

test_data[col]=[x[1]>0.5 for x in clf.predict_proba(test_data[X.columns])]

importance = clf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
    if v>0.0001:
        print('{}: {}'.format(X.columns[i],round(v,4)))

Гепатит: 0.0345
Бронжиальная астма: 0.0282
Регулярный прим лекарственных средств: 0.7434
Статус Курения: 0.036
Алкоголь: 0.0275
просыпание: 0.0106
вдовцы: 0.1197


In [196]:
# z4
col='z_4'

clf=RandomForestClassifier(max_depth=4,n_estimators=20,random_state=22, class_weight = 'balanced') #z4 0.5245

clf.fit(X,Y[col])

test_data[col]=[x[1]>0.5 for x in clf.predict_proba(test_data[X.columns])]

importance = clf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
    if v>0.0001:
        print('{}: {}'.format(X.columns[i],round(v,4)))

Пол: 0.0035
Религия: 0.0029
Образование: 0.0322
Профессия: 0.04
Вы работаете?: 0.1149
Выход на пенсию: 0.0784
Прекращение работы по болезни: 0.006
Сахарный диабет: 0.0124
Гепатит: 0.0071
Онкология: 0.0177
Хроническое заболевание легких: 0.0198
Бронжиальная астма: 0.0242
Регулярный прим лекарственных средств: 0.1706
Травмы за год: 0.0026
Переломы: 0.0173
Статус Курения: 0.0165
Возраст курения: 0.0547
Сигарет в день: 0.0481
Пассивное курение: 0.0115
Частота пасс кур: 0.0002
Алкоголь: 0.0281
Возраст алког: 0.0686
Сон после обеда: 0.0022
Спорт, клубы: 0.0004
Религия, клубы: 0.0106
засыпание: 0.0256
просыпание: 0.0197
сон_часы: 0.0682
В_БРАКЕ: 0.003
вдовцы: 0.05
не_был: 0.0042
Развод: 0.0286
Граждан: 0.0103


In [197]:
# z5
col='z_5'

clf = DecisionTreeClassifier(max_depth=4,
                            criterion='gini',
                            class_weight='balanced',
                                max_features='sqrt')

clf.fit(X,Y[col])

test_data[col]=[x[1]>0.6 for x in clf.predict_proba(test_data[X.columns])]

importance = clf.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    if v>0:
        print('{}: {}'.format(X.columns[i],round(v,4)))

Профессия: 0.011
Регулярный прим лекарственных средств: 0.4064
Травмы за год: 0.0251
Сон после обеда: 0.0121
Спорт, клубы: 0.0449
просыпание: 0.0829
сон_часы: 0.2485
вдовцы: 0.0374
Развод: 0.1316


In [201]:
result=test_data[['z_1','z_2','z_3','z_4','z_5']].rename(columns=dict(zip(zcol_z01,zcol)))